In [3]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate(
    [
        (
            "system",
            (
                "You are a helpful assistant. Answer questions using only the"
                " following context. If you don't know the answer just say you"
                " don't know, don't make it up:\n\n{context}"
            ),
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
)

result = chain.invoke("Describe Victory Mansions")
print(result.content)

Victory Mansions is a building where Winston Smith resides. It has glass doors that let in gritty dust, and the hallway smells of boiled cabbage and old rag mats. The flat is seven flights up, and the building has a faulty lift due to the electricity being cut off during daylight hours as part of the economy drive for Hate Week. Inside the flat, there is a telescreen that cannot be completely shut off, and a fruity voice reading out figures related to pig-iron production. The building is adorned with a large poster depicting an enormous face with the caption "BIG BROTHER IS WATCHING YOU."
